In [1]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from sklearn import linear_model
import sklearn
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge

# Generating Synthetic Data

In [2]:
def generateX(n,rho,p):
    cov_matrix = rho*np.ones((p,p))
    cov_matrix = cov_matrix + (1-rho)*np.identity(p)
    X = np.random.multivariate_normal(np.zeros(p), cov_matrix, n)
    return(X)

In [3]:
def generateStrongSignal(p,k):
    beta = np.zeros(p)
    for i in range(k):
        beta[i] = 1.0
    return(beta)

In [4]:
def generateWeakSignal(p,k):
    beta = np.zeros(p)
    for i in range(k):
        beta[i] = 1.0
    return(0.01*beta)

In [5]:
def generateVariedSignal(p,k):
    beta = np.zeros(p)
    for i in range(k):
        beta[i] = (i+1)*1.0/k
    return(beta)

In [6]:
'''
type_of_signal: 1. weak, 2. strong, 3. varied
'''

def generateBeta(p,k,type_of_signal):
    if type_of_signal not in [1,2,3]:
        print("Signal Error")
        return(None)
    if type_of_signal == 1:
        return(generateWeakSignal(p,k))
    elif type_of_signal == 2:
        return(generateStrongSignal(p,k))
    else:
        return(generateVariedSignal(p,k))

In [61]:
def generateY(X,beta):
    n,p = X.shape
#     epsilon = np.random.multivariate_normal(np.zeros(n), np.identity(n)*beta[0]*0.01)
    epsilon = np.random.multivariate_normal(np.zeros(n), np.identity(n))
    y = X.dot(beta) + epsilon
    return(y)

In [53]:
p = 1000
# NS = [1000,5000,10000]
# NS = [500,1000]
NS = [1000]

In [130]:
%%time

k = 100
signal_type = 3
# n = 10000
# num_batch = int(n/1000)
n = 1000
num_batch = 1
rho = 0

true_beta = generateBeta(p,k,signal_type)
Xs = []
ys = []
for i in range(num_batch):
    Xi = generateX(1000,rho,p)
    Xs.append(Xi)
    ys.append(generateY(Xi,true_beta).reshape((1000,1)))
X = np.vstack(Xs)
y = np.vstack(ys)

omp = OrthogonalMatchingPursuit(n_nonzero_coefs=k)
omp.fit(X, y)
coef = omp.coef_
min_offline_regret = np.sum(np.square(X.dot(coef)-y))

CPU times: user 1.37 s, sys: 42.5 ms, total: 1.42 s
Wall time: 726 ms


In [47]:
X.shape, y.shape

((1000, 1000), (1000, 1))

In [106]:
s = []
for i in range(50):
    k = 10
    signal_type = 1
    n = 1000
    num_batch = 1
    rho = 0

    true_beta = generateBeta(p,k,signal_type)
    X = generateX(1000,rho,p)
    y = generateY(X,true_beta).reshape((1000,1))

    s.append(run_Lasso(X,y,true_beta,k))

np.mean(s)

0.0

In [107]:
s = []
for i in range(50):
    k = 100
    signal_type = 2
    n = 1000
    num_batch = 1
    rho = 0

    true_beta = generateBeta(p,k,signal_type)
    X = generateX(1000,rho,p)
    y = generateY(X,true_beta).reshape((1000,1))

    s.append(run_Lasso(X,y,true_beta,k))
    
np.mean(s)

0.44880000000000003

# Model Implementation

## Implementing OMP

In [94]:
'''
Orthogonal Matching Pursuit ALGORITHM
'''
def run_omp(X,y,true_beta,k):
    start = timer()

    n, p = X.shape    
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=k)
    omp.fit(X, y)
    beta = omp.coef_
    
    ypred = omp.predict(X)    
    squared_error_sum = np.sum(np.square(ypred-y))
    
    end = timer()
    Time = end-start
    
    RMSE = np.sqrt(squared_error_sum*1.0/n)
    DR = (np.sum((np.multiply(true_beta,beta)!=0)*1.0))*1.0/k
    
    SS_tot = np.sum(np.square(y-np.mean(y)))
    Rsquared = 1-squared_error_sum*1.0/SS_tot

    print(RMSE,DR,Rsquared,Time,'OMP')    
    return(RMSE,DR,Rsquared,Time,squared_error_sum*1.0/n)

In [44]:
# k=10
run_omp(X,y,true_beta,k)

139.44754343017448 1.0 -1998.074455957402 0.025807139929383993 OMP


(139.44754343017448,
 1.0,
 -1998.074455957402,
 0.025807139929383993,
 19445.6173687104)

In [101]:
# k=100
run_omp(X,y,true_beta,k)

433.123836440686 1.0 -1989.4969425684103 0.06176298903301358 OMP


(433.123836440686,
 1.0,
 -1989.4969425684103,
 0.06176298903301358,
 187596.2576930981)

## Implementing LASSO

In [122]:
'''
LASSO ALGORITHM
'''
def run_Lasso(X,y,true_beta,k,CURRENT_RANDOM_SEED=1):
    start = timer()
    n, p = X.shape
    
    lasso_model = linear_model.Lasso(alpha = 1.0,random_state=CURRENT_RANDOM_SEED)
    lasso_model.fit(X,y)
    beta = lasso_model.coef_
    ypred = lasso_model.predict(X)
    
    squared_error_sum = np.sum(np.square(ypred-y))
    
    end = timer()
    Time = end-start
    
    RMSE = np.sqrt(squared_error_sum*1.0/n)
    DR = (np.sum((np.multiply(true_beta,beta)!=0)*1.0))*1.0/k
    
    SS_tot = np.sum(np.square(y-np.mean(y)))
    Rsquared = 1-squared_error_sum*1.0/SS_tot
    
    print(RMSE,DR,Rsquared,Time,'LASSO')    
    return(DR)

In [125]:
#k=10,strong

run_Lasso(X,y,true_beta,k)

103.62166773997666 0.4 -1000.7139984445095 0.02444210695102811 LASSO


0.4

In [123]:
#k=10,weak

run_Lasso(X,y,true_beta,k)

32.04898703665147 0.0 -999.0000000000002 0.030062678968533874 LASSO


0.0

In [127]:
#k=10,varied

run_Lasso(X,y,true_beta,k)

66.39607461195448 0.0 -998.9999999999999 0.020445049973204732 LASSO


0.0

In [79]:
#k=100

run_Lasso(X,y,true_beta,k)

308.02388535513546 0.33 -1023.7758201939996 0.02579942694865167 LASSO


()

## Implementing TSGD

In [29]:
'''
Truncation function for TSGD Algorithm
'''
def truncate(beta,k):
    p = beta.shape[0]
    sorted_indices = np.abs(beta).argsort()[::-1].tolist()
    dummy = np.zeros(p)
    for el in sorted_indices[:k]:
        dummy[el] = 1.0
    return(np.multiply(beta,dummy))

In [24]:
'''
TSGD ALGORITHM
'''
def run_TSGD(X,y,true_beta,k,min_offline_errors):
    start = timer()

    n, p = X.shape
    ns = NS
    ns = [nn-1 for nn in ns]
    n_idx = 0
    eta = np.log(ns[n_idx])*1.0/ns[n_idx]

    squared_error_sum = 0
    true_support_sum = 0

    beta = np.random.rand(p)-0.5
    for i in range(n):
        prev_beta = beta
        y_pred = X[i,:].dot(prev_beta)
        
        loss_i = (y_pred-y[i,])**2
        current_detection_rate = (np.sum((np.multiply(true_beta,prev_beta)!=0)*1.0))*1.0/k
        squared_error_sum += loss_i
        true_support_sum += current_detection_rate
        
        beta = prev_beta + eta*(y[i]-np.dot(X[i,:],prev_beta))*X[i,:]
        beta = truncate(beta,k)

        if i in ns:
            end = timer()
            Time = end-start
            
            RMSE = np.sqrt(squared_error_sum*1.0/i)
            DR = true_support_sum*1.0/i

            SS_tot = np.sum(np.square(y[:i+1]-np.mean(y[:i+1])))
            Rsquared = 1-squared_error_sum*1.0/SS_tot
            
            regret = squared_error_sum*1.0/i - min_offline_errors[n_idx]

            if n_idx != len(ns) - 1:
                n_idx += 1
                eta = np.log(ns[n_idx])*1.0/ns[n_idx]

            print(RMSE[0],DR,Rsquared[0],Time,regret[0],i+1,'TSGD')
    
    return()

In [71]:
#k=5
run_TSGD(X,y,true_beta,k,min_offline_errors)

2.3411812159035787 0.15355355355355338 0.024366969131436433 0.10970593197271228 -19440.1362392247 1000 TSGD


()

In [129]:
#k=10
run_TSGD(X,y,true_beta,k,min_offline_errors)

2.0800460568006334 0.16876876876877026 0.10021613326151524 0.12195924506522715 -19441.290777111986 1000 TSGD


()

In [134]:
#k=100
run_TSGD(X,y,true_beta,k,min_offline_errors)

7.804599320867919 0.14143143143143186 -0.7576794697734834 0.11690987995825708 -19384.705598151108 1000 TSGD


()

## Implementing OLST

In [34]:
'''
OLST ALGORITHM
'''
def run_OLST(X,y,true_beta,k,rho,min_offline_errors):
    start = timer()
    n, p = X.shape
    ns = NS
    ns = [nn-1 for nn in ns]
    n_idx = 0
    y = y.reshape((n,1))

    squared_error_sum = 0
    true_support_sum = 0

    beta = np.random.rand(p)-0.5

    #INITIALIZATION
    mu_x = np.zeros((p))
    mu_y = 0
    S_xx = np.zeros((p,p))
    S_xy = np.zeros((p,1))

    for i in range(n):
        if i%100==1:
            print(i)

        prev_beta = beta
        y_pred = X[i,:].dot(prev_beta)
        loss_i = (y_pred-y[i,0])**2
        current_detection_rate = (np.sum((np.multiply(true_beta,prev_beta)!=0)*1.0))*1.0/k
        
#         print(loss_i,current_detection_rate)
        
        squared_error_sum += loss_i
        true_support_sum += current_detection_rate
                    
        # Finding $\hat{beta}$ by OLS
        X_current, y_current = X[i,:].reshape((1,p)), y[i]
        mu_x = i*mu_x/(i+1) + X_current*1.0/(i+1)
        mu_y = i*mu_y/(i+1) + y_current*1.0/(i+1)
        S_xx = S_xx*i/(i+1) + np.dot(X_current.T,X_current)*1.0/(i+1)
        S_xy = S_xy*i/(i+1) + y_current*X_current.T*1.0/(i+1)
    
        # Normalization
        if i>=1:
            Pi = np.linalg.inv(np.sqrt(np.multiply((S_xx - np.square(mu_x)),np.identity(p))))
            S_xx_n = np.dot(Pi, S_xx-np.dot(mu_x,mu_x.T))
            S_xx_n = np.dot(S_xx_n,Pi)

            S_xy_n = np.dot(Pi, S_xy) - mu_y*np.dot(Pi, mu_x.T).reshape((p,1))

        clf = Ridge(alpha=0.001)
        if k==10:
            clf = Ridge(alpha=0.001)
        if k==100 and rho!=0:
            clf = Ridge(alpha=1.0)
        
        if i==0:
            clf.fit(S_xx,S_xy)
        else:
            clf.fit(S_xx_n,S_xy_n)
        beta = clf.coef_.reshape((p,))

        # Keeping only k variables with largest $|\hat{\beta}_j|$
        sorted_indices = np.abs(beta).argsort()[::-1].tolist()
        k_biggest_indices = np.sort(sorted_indices[:k])
        
        # Fitting the model on the selected features by OLS
        selected_X = X[:i+1,k_biggest_indices]

        clf = Ridge(alpha=0.0001)
        if k==10:
            clf = Ridge(alpha=0.0001)
        if k==100 and rho!=0:       
            clf = Ridge(alpha=0.1)
        clf.fit(selected_X, y[:i+1])
        OLS_beta = clf.coef_.reshape((k,))
        
#         OLS_beta = np.dot(np.linalg.inv(np.dot(selected_X.T,selected_X)),np.dot(selected_X.T,y_current))
                
        beta = np.zeros(p)
        for j,ind in enumerate(k_biggest_indices):
            beta[ind] = OLS_beta[j]

        if i in ns:
            end = timer()
            Time = end-start
            
            RMSE = np.sqrt(squared_error_sum*1.0/i)
            DR = true_support_sum*1.0/i

            SS_tot = np.sum(np.square(y[:i+1]-np.mean(y[:i+1])))
            Rsquared = 1-squared_error_sum*1.0/SS_tot

            regret = squared_error_sum*1.0/n - min_offline_errors[n_idx]

            if n_idx != len(ns) - 1:
                n_idx += 1

            print(RMSE,DR,Rsquared,Time,regret,i+1,'OLST')
    
    return()

In [43]:
%%time
# k=10
run_OLST(X,y,true_beta,k,rho,min_offline_errors)

1


/Users/fransiscasusan/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 2.355448258192828e-21
  ' condition number: {}'.format(rcond), RuntimeWarning)
/Users/fransiscasusan/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 2.131469109782073e-18
  ' condition number: {}'.format(rcond), RuntimeWarning)
/Users/fransiscasusan/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 3.407537493882403e-17
  ' condition number: {}'.format(rcond), RuntimeWarning)


101
201
301
401
501
601
701
801
901
2.026001052106114 0.9207207207207206 0.5784471252822158 217.18382744397968 -19441.516793127525 1000 OLST
CPU times: user 8min 51s, sys: 23.3 s, total: 9min 15s
Wall time: 3min 37s


()

In [35]:
%%time
# k=100
run_OLST(X,y,true_beta,k,rho,min_offline_errors)

1


/Users/fransiscasusan/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 6.90924838998484e-21
  ' condition number: {}'.format(rcond), RuntimeWarning)
/Users/fransiscasusan/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 9.012674642946503e-20
  ' condition number: {}'.format(rcond), RuntimeWarning)
/Users/fransiscasusan/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 1.5317212493085947e-17
  ' condition number: {}'.format(rcond), RuntimeWarning)


101
201
301
401
501
601
701
801
901
10.881980546689126 0.61242242242242 -0.15422203613212915 220.0004774958361 -204857.2724323871 1000 OLST
CPU times: user 8min 59s, sys: 29.5 s, total: 9min 29s
Wall time: 3min 40s


()

## Implementing OLST-TSGD

In [ ]:
'''
Hybrid ALGORITHM
'''
def run_hybrid(X,y,true_beta,k,rho,min_offline_errors,BURNIN):
    start = timer()
    n, p = X.shape
    ns = NS
    ns = [nn-1 for nn in ns]
    n_idx = 0
    eta = np.log(ns[n_idx])*1.0/ns[n_idx]
    y = y.reshape((n,1))

    squared_error_sum = 0
    true_support_sum = 0

    beta = np.random.rand(p)-0.5

    #INITIALIZATION
    mu_x = np.zeros(p)
    mu_y = 0
    S_xx = np.zeros((p,p))
    S_xy = np.zeros((p,1))

    for i in range(BURNIN):

        prev_beta = beta
        y_pred = X[i,:].dot(prev_beta)
        loss_i = (y_pred-y[i,0])**2
        current_detection_rate = (np.sum((np.multiply(true_beta,prev_beta)!=0)*1.0))*1.0/k
        
        squared_error_sum += loss_i
        true_support_sum += current_detection_rate
        
        # Finding $\hat{beta}$ by OLS
        X_current, y_current = X[i,:].reshape((1,p)), y[i]
        mu_x = i*mu_x/(i+1) + X_current*1.0/(i+1)
        mu_y = i*mu_y/(i+1) + y_current*1.0/(i+1)
        S_xx = S_xx*i/(i+1) + np.dot(X_current.T,X_current)*1.0/(i+1)
        S_xy = S_xy*i/(i+1) + y_current*X_current.T*1.0/(i+1)
    
        # Normalization
        if i>=1:
            Pi = np.linalg.inv(np.sqrt(np.multiply((S_xx - np.square(mu_x)),np.identity(p))))
            S_xx_n = np.dot(Pi, S_xx-np.dot(mu_x,mu_x.T))
            S_xx_n = np.dot(S_xx_n,Pi)

            S_xy_n = np.dot(Pi, S_xy) - mu_y*np.dot(Pi, mu_x.T).reshape((p,1))


        clf = Ridge(alpha=0.001)
        if k==10:
            clf = Ridge(alpha=0.001)
        if k==100 and rho!=0:
            clf = Ridge(alpha=1.0)
        
        if i==0:
            clf.fit(S_xx,S_xy)
        else:
            clf.fit(S_xx_n, S_xy_n)
        beta = clf.coef_.reshape((p,))

        # Keeping only k variables with largest $|\hat{\beta}_j|$
        sorted_indices = np.abs(beta).argsort()[::-1].tolist()
        k_biggest_indices = np.sort(sorted_indices[:k])
        
        # Fitting the model on the selected features by OLS
        selected_X = X[:i+1,k_biggest_indices]

        clf = Ridge(alpha=0.0001)
        if k==10:
            clf = Ridge(alpha=0.0001)
        if k==100 and rho!=0:       
            clf = Ridge(alpha=0.1)
        clf.fit(selected_X, y[:i+1])
        OLS_beta = clf.coef_.reshape((k,))
        
#         OLS_beta = np.dot(np.linalg.inv(np.dot(selected_X.T,selected_X)),np.dot(selected_X.T,y_current))
                
        beta = np.zeros(p)
        for j,ind in enumerate(k_biggest_indices):
            beta[ind] = OLS_beta[j]

        if i in ns:
            end = timer()
            Time = end-start
            
            RMSE = np.sqrt(squared_error_sum*1.0/i)
            DR = true_support_sum*1.0/i

            SS_tot = np.sum(np.square(y[:i+1]-np.mean(y[:i+1])))
            Rsquared = 1-squared_error_sum*1.0/SS_tot

            regret = squared_error_sum*1.0/n - min_offline_errors[n_idx]

            if n_idx != len(ns) - 1:
                n_idx += 1
                eta = np.log(ns[n_idx])*1.0/ns[n_idx]

            print(RMSE,DR,Rsquared,Time,regret,i+1,'OLST-hybrid')

    y = y.reshape((n,))
    for i in range(BURNIN, n):

        prev_beta = beta
        y_pred = X[i,:].dot(prev_beta)
        
        loss_i = (y_pred-y[i,])**2
        current_detection_rate = (np.sum((np.multiply(true_beta,prev_beta)!=0)*1.0))*1.0/k
        squared_error_sum += loss_i
        true_support_sum += current_detection_rate
        
        beta = prev_beta + eta*(y[i]-np.dot(X[i,:],prev_beta))*X[i,:]
        beta = truncate(beta,k)

        if i in ns:
            end = timer()
            Time = end-start
            
            RMSE = np.sqrt(squared_error_sum*1.0/i)
            DR = true_support_sum*1.0/i

            SS_tot = np.sum(np.square(y[:i+1]-np.mean(y[:i+1])))
            Rsquared = 1-squared_error_sum*1.0/SS_tot
            
            regret = squared_error_sum*1.0/i - min_offline_errors[n_idx]

            if n_idx != len(ns) - 1:
                n_idx += 1
                eta = np.log(ns[n_idx])*1.0/ns[n_idx]

            print(RMSE,DR,Rsquared,Time,regret,i+1,'TSGD-hybrid')
    
    return()

In [ ]:
run_hybrid(X,y,true_beta,k,rho,min_offline_errors,50)

## COMPARISON

In [42]:
min_offline_errors = []
for n in NS:
    Xn        = X[:n,:]
    yn         = y[:n]
    res_omp = run_omp(Xn,yn,true_beta,k)
    min_offline_errors.append(res_omp[-1])

139.44754343017448 1.0 -1998.074455957402 0.029607095988467336 OMP


In [ ]:
for n in NS:
    Xn          = X[:n,:]
    yn          = y[:n]
    run_Lasso(Xn,yn,true_beta,k)

In [ ]:
run_TSGD(X,y,true_beta,k,min_offline_errors)

In [ ]:
run_OLST(X,y,true_beta,k,rho,min_offline_errors)

In [ ]:
run_hybrid(X,y,true_beta,k,rho,min_offline_errors,50)

In [ ]:
run_hybrid(X,y,true_beta,k,rho,min_offline_errors,500)